# 🦅 EagleEye - Exploratory Data Analysis

**Deloitte x AUC Hackathon 2026** | Fresh Flow Markets: Inventory Management

This notebook explores the dataset to understand:
1. Data structure and quality
2. Sales patterns and demand drivers
3. Inventory-related insights
4. External factors impacting sales

## 1. Setup & Data Loading

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import os
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries loaded successfully!")

Libraries loaded successfully!


In [2]:
# Data directory
DATA_DIR = '../Data'

# List all CSV files
csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
print(f"Found {len(csv_files)} CSV files:")
for f in sorted(csv_files):
    size_mb = os.path.getsize(os.path.join(DATA_DIR, f)) / (1024 * 1024)
    print(f"  - {f}: {size_mb:.2f} MB")

Found 19 CSV files:
  - dim_add_ons.csv: 1.48 MB
  - dim_bill_of_materials.csv: 0.00 MB
  - dim_campaigns.csv: 0.05 MB
  - dim_items.csv: 13.80 MB
  - dim_menu_item_add_ons.csv: 0.12 MB
  - dim_menu_items.csv: 2.04 MB
  - dim_places.csv: 2.01 MB
  - dim_skus.csv: 0.00 MB
  - dim_stock_categories.csv: 0.00 MB
  - dim_taxonomy_terms.csv: 0.05 MB
  - dim_users.csv: 11.00 MB
  - fct_bonus_codes.csv: 0.00 MB
  - fct_campaigns.csv: 0.14 MB
  - fct_cash_balances.csv: 358.51 MB
  - fct_inventory_reports.csv: 0.00 MB
  - fct_invoice_items.csv: 0.21 MB
  - fct_order_items.csv: 211.31 MB
  - fct_orders.csv: 62.16 MB
  - most_ordered.csv: 4.06 MB


In [3]:
# Load all datasets
datasets = {}

for f in csv_files:
    name = f.replace('.csv', '')
    path = os.path.join(DATA_DIR, f)
    try:
        datasets[name] = pd.read_csv(path, low_memory=False)
        print(f"✓ Loaded {name}: {datasets[name].shape[0]:,} rows, {datasets[name].shape[1]} columns")
    except Exception as e:
        print(f"✗ Error loading {name}: {e}")

✓ Loaded most_ordered: 95,435 rows, 7 columns
✓ Loaded fct_campaigns: 641 rows, 55 columns
✓ Loaded fct_invoice_items: 3,124 rows, 8 columns
✓ Loaded dim_skus: 4 rows, 11 columns


✓ Loaded fct_orders: 399,810 rows, 40 columns
✓ Loaded dim_places: 1,824 rows, 189 columns


✓ Loaded dim_items: 87,713 rows, 36 columns


✓ Loaded fct_order_items: 1,999,341 rows, 23 columns


✓ Loaded dim_users: 22,955 rows, 41 columns
✓ Loaded fct_bonus_codes: 6 rows, 9 columns
✓ Loaded dim_taxonomy_terms: 904 rows, 6 columns
✓ Loaded dim_campaigns: 641 rows, 7 columns
✓ Loaded fct_inventory_reports: 0 rows, 10 columns
✓ Loaded dim_add_ons: 21,101 rows, 12 columns
✓ Loaded dim_bill_of_materials: 2 rows, 7 columns
✓ Loaded dim_stock_categories: 3 rows, 6 columns
✓ Loaded dim_menu_item_add_ons: 2,459 rows, 8 columns
✓ Loaded dim_menu_items: 30,407 rows, 11 columns


✓ Loaded fct_cash_balances: 52,915 rows, 12 columns


## 2. Data Overview & Quality Assessment

In [4]:
# Summary of all datasets
summary_data = []
for name, df in datasets.items():
    summary_data.append({
        'Dataset': name,
        'Rows': df.shape[0],
        'Columns': df.shape[1],
        'Missing (%)': (df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100).round(2),
        'Memory (MB)': (df.memory_usage(deep=True).sum() / (1024**2)).round(2)
    })

summary_df = pd.DataFrame(summary_data).sort_values('Rows', ascending=False)
summary_df.style.background_gradient(subset=['Rows', 'Memory (MB)'], cmap='Blues')

,Dataset,Rows,Columns,Missing (%),Memory (MB)
7,fct_order_items,1999341,23,24.830000,740.700000
4,fct_orders,399810,40,37.910000,278.050000
0,most_ordered,95435,7,20.050000,14.130000
6,dim_items,87713,36,26.150000,54.740000
18,fct_cash_balances,52915,12,1.250000,315.880000
17,dim_menu_items,30407,11,0.000000,6.990000
8,dim_users,22955,41,34.160000,29.170000
13,dim_add_ons,21101,12,0.000000,3.930000
2,fct_invoice_items,3124,8,0.830000,0.370000
16,dim_menu_item_add_ons,2459,8,0.000000,0.380000


In [5]:
# Detailed look at key tables
key_tables = ['fct_orders', 'fct_order_items', 'dim_items', 'dim_places']

for table in key_tables:
    if table in datasets:
        print(f"\n{'='*60}")
        print(f"📊 {table.upper()}")
        print(f"{'='*60}")
        print(f"\nShape: {datasets[table].shape}")
        print(f"\nColumns: {list(datasets[table].columns)}")
        print(f"\nData Types:\n{datasets[table].dtypes}")
        print(f"\nFirst 3 rows:")
        display(datasets[table].head(3))


📊 FCT_ORDERS

Shape: (399810, 40)

Columns: ['id', 'user_id', 'created', 'updated', 'updated_by', 'account_id', 'cash_amount', 'cashier_notified', 'channel', 'customer_mobile_phone', 'customer_name', 'code', 'delivery_charge', 'delivery_location_id', 'demo_mode', 'discount_amount', 'driver_id', 'external_id', 'instructions', 'items_amount', 'payment_method', 'pickup_time', 'place_id', 'points_earned', 'points_redeemed', 'promise_time', 'rejection_reason', 'referring_user_id', 'service_charge', 'source', 'split_bill', 'split_bill_type', 'status', 'synchronized_to_accounting', 'table_id', 'tier_id', 'total_amount', 'trainee_mode', 'type', 'vat_amount']

Data Types:
id                              int64
user_id                         int64
created                         int64
updated                         int64
updated_by                    float64
account_id                    float64
cash_amount                   float64
cashier_notified              float64
channel                

,id,user_id,created,updated,updated_by,account_id,cash_amount,cashier_notified,channel,customer_mobile_phone,customer_name,code,delivery_charge,delivery_location_id,demo_mode,discount_amount,driver_id,external_id,instructions,items_amount,payment_method,pickup_time,place_id,points_earned,points_redeemed,promise_time,rejection_reason,referring_user_id,service_charge,source,split_bill,split_bill_type,status,synchronized_to_accounting,table_id,tier_id,total_amount,trainee_mode,type,vat_amount
0,60825,512,1613139429,1687470923,NaN,NaN,50.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,50.0,Counter,NaN,59897.0,25.0,0.0,1.613140e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59898.0,50.0,NaN,Takeaway,12.50
1,60841,514,1613144783,1687470923,NaN,NaN,655.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,655.0,Counter,NaN,59821.0,327.0,0.0,1.613146e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,655.0,NaN,Takeaway,163.75
2,60852,489,1613149610,1687470923,NaN,NaN,430.0,NaN,App,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,430.0,Counter,NaN,59821.0,215.0,0.0,1.613151e+09,NaN,NaN,NaN,Customer,NaN,NaN,Closed,NaN,NaN,59822.0,430.0,NaN,Takeaway,107.50



📊 FCT_ORDER_ITEMS

Shape: (1999341, 23)

Columns: ['id', 'user_id', 'created', 'updated', 'title', 'campaign_id', 'commission_amount', 'cost', 'discount_amount', 'external_id', 'group', 'instructions', 'item_id', 'order_id', 'points_earned', 'points_redeemed', 'price', 'quantity', 'redemptions', 'removed_ingredients', 'add_on_ids', 'status', 'vat_amount']

Data Types:
id                       int64
user_id                  int64
created                  int64
updated                  int64
title                      str
campaign_id            float64
commission_amount      float64
cost                   float64
discount_amount        float64
external_id                str
group                  float64
instructions               str
item_id                  int64
order_id               float64
points_earned          float64
points_redeemed        float64
price                  float64
quantity                 int64
redemptions              int64
removed_ingredients        str
add_on_i

,id,user_id,created,updated,title,campaign_id,commission_amount,cost,discount_amount,external_id,group,instructions,item_id,order_id,points_earned,points_redeemed,price,quantity,redemptions,removed_ingredients,add_on_ids,status,vat_amount
0,60824,512,1613139429,1708921636,Latte,NaN,1.50,50.0,0.0,NaN,NaN,NaN,60822,60825.0,NaN,NaN,25.0,2,0,NaN,NaN,Pending,NaN
1,60837,514,1613144783,1708921895,The Classic Meal,NaN,5.25,175.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,175.0,1,0,NaN,59995,Pending,NaN
2,60838,514,1613144783,1708921895,The Classic Meal,NaN,4.80,160.0,0.0,NaN,NaN,NaN,59999,60841.0,NaN,NaN,160.0,1,0,NaN,NaN,Pending,NaN



📊 DIM_ITEMS

Shape: (87713, 36)

Columns: ['id', 'user_id', 'created', 'updated', 'accounting_reference', 'barcode', 'deleted', 'delivery', 'demo_mode', 'description', 'discountable', 'display_for_customers', 'eat_in', 'external_id', 'image', 'index', 'number', 'price', 'purchases', 'removable_ingredients', 'section_id', 'status', 'takeaway', 'title', 'trainee_mode', 'type', 'variable_price', 'vat', 'voucher', 'add_on_category_ids', 'printer_category_ids', 'standard_voucher_validity', 'manage_inventory', 'all_you_can_eat', 'all_you_can_eat_item_ids', 'all_you_can_eat_validity']

Data Types:
id                             int64
user_id                        int64
created                        int64
updated                        int64
accounting_reference         float64
barcode                          str
deleted                        int64
delivery                       int64
demo_mode                      int64
description                      str
discountable                   

,id,user_id,created,updated,accounting_reference,barcode,deleted,delivery,demo_mode,description,discountable,display_for_customers,eat_in,external_id,image,index,number,price,purchases,removable_ingredients,section_id,status,takeaway,title,trainee_mode,type,variable_price,vat,voucher,add_on_category_ids,printer_category_ids,standard_voucher_validity,manage_inventory,all_you_can_eat,all_you_can_eat_item_ids,all_you_can_eat_validity
0,59837,423,1610874221,1719416473,NaN,NaN,0,1,0,"Burger with beyond meat patty, tomato, red oni...",1,0,1,NaN,NaN,1,NaN,110.0,30.0,NaN,59827.0,Active,1,The Classic,0.0,Normal,0,25.0,0,61551|61554|61562|61571|61575|61580,NaN,NaN,0,0,NaN,NaN
1,59838,423,1610874348,1719416473,NaN,NaN,0,1,0,Burger with baked portobello mushroom with gar...,1,0,1,NaN,NaN,2,NaN,110.0,1.0,NaN,59827.0,Active,1,The Rudimental,0.0,Normal,0,25.0,0,61551|61554|61562|61571|61575|61580,NaN,NaN,0,0,NaN,NaN
2,59839,423,1610874458,1719416473,NaN,NaN,0,1,0,"Burger with Celeriach (poached in stock), toma...",1,0,1,NaN,NaN,3,NaN,110.0,1.0,NaN,59827.0,Active,1,The Garden,0.0,Normal,0,25.0,0,61551|61554|61562|61571|61575|61580,NaN,NaN,0,0,NaN,NaN



📊 DIM_PLACES

Shape: (1824, 189)

Columns: ['id', 'user_id', 'created', 'updated', 'title', 'activated', 'active', 'bankrupt', 'binding_period', 'chain_id', 'contact_email', 'contact_mobile_phone', 'contract_start', 'country', 'currency', 'cvr_number', 'duplicate', 'ecommerce_fee', 'eu_commission', 'invoicing_start_date', 'isv_commission', 'isv_partner', 'non_eu_commission', 'onboarded_by', 'payment_terminal_provider', 'area_id', 'processing_fee', 'sales_outcome_id', 'sales_stage', 'service_charge', 'setup_type', 'termination_date', 'termination_value', 'transferred_contract', 'type_id', 'cashier_ids', 'manager_ids', 'driver_ids', 'payment_terminal_ids', 'kitchen_staff_ids', 'contact_name', 'description', 'cuisine_ids', 'trainee_mode', 'demo_mode', 'default_order_type_cashier', 'default_order_type_customer', 'show_bestsellers', 'use_quick_search', 'display_cashier_images', 'display_created_by', 'use_customer_facing_display', 'timezone', 'contact_language', 'business_name', 'area', 'st

,id,user_id,created,updated,title,activated,active,bankrupt,binding_period,chain_id,contact_email,contact_mobile_phone,contract_start,country,currency,cvr_number,duplicate,ecommerce_fee,eu_commission,invoicing_start_date,isv_commission,isv_partner,non_eu_commission,onboarded_by,payment_terminal_provider,...,protect_sensitive_sections,rating,receipt_printing_enabled,receive_phone_calls,skip_order_confirmation,slot_duration,sms_balance,sms_on_ready_text,suppress_print_receipt_prompt,table_bookings,takeaway,takeaway_qr_code_link,terminal_provider,theme,tiered_loyalty_program,vat,vip_threshold,visit_duration,votes,waiting_time,unspecified_section_id,unspecified_item_id,enable_sms_on_order,sms_on_order_text,exclude_from_ops_report
0,59813,417,1610718144,1744221291,Green Love,0,0,0,36,NaN,place_59813@example.com,4559813,NaN,DK,DKK,30025334,0,0.0,0.0,NaN,2.0,0,0.0,lovingloyalty,NaN,...,1,198.0,1,1,0,30,0,NaN,1,0,1,NaN,NaN,NaN,0,25.0,20,90,22,25,5972947,306775,0,NaN,0
1,59817,417,1610718304,1744221291,Grønnere,0,0,0,36,NaN,place_59817@example.com,4559817,NaN,DK,DKK,41235586,0,0.0,0.0,NaN,2.0,0,0.0,lovingloyalty,NaN,...,1,0.0,0,0,0,30,0,NaN,1,0,1,NaN,NaN,NaN,0,25.0,20,90,0,15,5972948,306777,0,NaN,0
2,59821,417,1610866418,1744221291,Eden Jaxx,0,0,0,36,NaN,place_59821@example.com,4559821,NaN,DK,DKK,41465549,0,0.0,0.0,NaN,2.0,0,0.0,lovingloyalty,NaN,...,1,75.0,0,1,0,30,0,NaN,1,0,1,NaN,NaN,NaN,0,25.0,20,90,8,25,5972949,306779,0,NaN,0


## 3. Orders Analysis

In [6]:
# Convert Unix timestamps to datetime
orders = datasets['fct_orders'].copy()
orders['created_dt'] = pd.to_datetime(orders['created'], unit='s')
orders['date'] = orders['created_dt'].dt.date
orders['hour'] = orders['created_dt'].dt.hour
orders['day_of_week'] = orders['created_dt'].dt.day_name()
orders['month'] = orders['created_dt'].dt.to_period('M')

print(f"Date Range: {orders['created_dt'].min()} to {orders['created_dt'].max()}")
print(f"Total Orders: {len(orders):,}")
print(f"Unique Places: {orders['place_id'].nunique()}")

Date Range: 2021-02-12 14:17:09 to 2024-02-16 19:14:57
Total Orders: 399,810
Unique Places: 359


In [7]:
# Orders over time
daily_orders = orders.groupby('date').size().reset_index(name='order_count')
daily_orders['date'] = pd.to_datetime(daily_orders['date'])

fig = px.line(daily_orders, x='date', y='order_count', 
              title='📈 Daily Order Volume Over Time',
              labels={'date': 'Date', 'order_count': 'Number of Orders'})
fig.update_layout(template='plotly_white')
fig.show()

In [8]:
# Orders by day of week
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_orders = orders.groupby('day_of_week').size().reindex(dow_order)

fig = px.bar(x=dow_orders.index, y=dow_orders.values,
             title='📅 Orders by Day of Week',
             labels={'x': 'Day', 'y': 'Number of Orders'},
             color=dow_orders.values,
             color_continuous_scale='Blues')
fig.update_layout(template='plotly_white', showlegend=False)
fig.show()

In [9]:
# Orders by hour
hourly_orders = orders.groupby('hour').size()

fig = px.bar(x=hourly_orders.index, y=hourly_orders.values,
             title='⏰ Orders by Hour of Day',
             labels={'x': 'Hour', 'y': 'Number of Orders'},
             color=hourly_orders.values,
             color_continuous_scale='Viridis')
fig.update_layout(template='plotly_white', showlegend=False)
fig.show()

## 4. Menu Items & Popularity Analysis

In [10]:
# Order items analysis
order_items = datasets['fct_order_items'].copy()

print(f"Total Order Items: {len(order_items):,}")
print(f"Unique Items: {order_items['item_id'].nunique()}")
print(f"\nOrder Item Columns: {list(order_items.columns)}")

Total Order Items: 1,999,341
Unique Items: 26036

Order Item Columns: ['id', 'user_id', 'created', 'updated', 'title', 'campaign_id', 'commission_amount', 'cost', 'discount_amount', 'external_id', 'group', 'instructions', 'item_id', 'order_id', 'points_earned', 'points_redeemed', 'price', 'quantity', 'redemptions', 'removed_ingredients', 'add_on_ids', 'status', 'vat_amount']


In [11]:
# Most ordered items
if 'most_ordered' in datasets:
    most_ordered = datasets['most_ordered'].copy()
    top_items = most_ordered.nlargest(20, 'order_count')
    
    fig = px.bar(top_items, x='order_count', y='item_name',
                 orientation='h',
                 title='🏆 Top 20 Most Ordered Items',
                 labels={'order_count': 'Order Count', 'item_name': 'Item'},
                 color='order_count',
                 color_continuous_scale='Oranges')
    fig.update_layout(template='plotly_white', yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [12]:
# Item price distribution
items = datasets['dim_items'].copy()
items_with_price = items[items['price'] > 0]

fig = px.histogram(items_with_price, x='price', nbins=50,
                   title='💰 Item Price Distribution (DKK)',
                   labels={'price': 'Price (DKK)', 'count': 'Number of Items'})
fig.update_layout(template='plotly_white')
fig.show()

print(f"\nPrice Statistics:")
print(items_with_price['price'].describe())


Price Statistics:
count    7.948900e+04
mean     1.450197e+03
std      3.551425e+05
min      1.000000e-02
25%      3.000000e+01
50%      6.000000e+01
75%      9.500000e+01
max      1.000030e+08
Name: price, dtype: float64


## 5. Revenue & Sales Analysis

In [13]:
# Revenue analysis
if 'total_amount' in orders.columns:
    orders['total_amount'] = pd.to_numeric(orders['total_amount'], errors='coerce')
    
    daily_revenue = orders.groupby('date').agg({
        'total_amount': 'sum',
        'id': 'count'
    }).reset_index()
    daily_revenue.columns = ['date', 'revenue', 'order_count']
    daily_revenue['date'] = pd.to_datetime(daily_revenue['date'])
    daily_revenue['avg_order_value'] = daily_revenue['revenue'] / daily_revenue['order_count']
    
    print(f"Total Revenue: {orders['total_amount'].sum():,.2f} DKK")
    print(f"Average Order Value: {orders['total_amount'].mean():.2f} DKK")
    
    fig = px.line(daily_revenue, x='date', y='revenue',
                  title='💵 Daily Revenue Over Time (DKK)',
                  labels={'date': 'Date', 'revenue': 'Revenue (DKK)'})
    fig.update_layout(template='plotly_white')
    fig.show()

Total Revenue: 178,090,176.50 DKK
Average Order Value: 445.44 DKK


## 6. Inventory-Related Insights

In [14]:
# Check inventory-related tables
inventory_tables = ['dim_skus', 'dim_stock_categories', 'fct_inventory_reports']

for table in inventory_tables:
    if table in datasets:
        print(f"\n{'='*40}")
        print(f"📦 {table}")
        print(f"{'='*40}")
        print(f"Shape: {datasets[table].shape}")
        if len(datasets[table]) > 0:
            display(datasets[table].head())
        else:
            print("⚠️ Empty dataset")


📦 dim_skus
Shape: (4, 11)


,id,user_id,created,updated,stock_category_id,item_id,title,quantity,low_stock_threshold,type,unit
0,1,424,1744481238,1744481238,1,66679.0,Fritz Kola,10.0,10.0,normal,pcs
1,2,424,1744481571,1744481571,2,NaN,Tomat,5.7,10.0,normal,kg
2,3,424,1744481596,1744481596,2,NaN,Agurk,-10.5,2.0,normal,kg
3,4,424,1744481634,1744481634,3,59913.0,Potatos,0.0,10.0,composite,pcs



📦 dim_stock_categories
Shape: (3, 6)


,id,user_id,created,updated,place_id,title
0,1,424,1744481238,1744481238,59897,Kolde Drikke
1,2,424,1744481508,1744481508,59897,Grøntsager
2,3,424,1744481634,1744481634,59897,Pizza



📦 fct_inventory_reports
Shape: (0, 10)
⚠️ Empty dataset


In [15]:
# Analyze demand variability per item (for forecasting)
item_demand = order_items.groupby('item_id').agg({
    'quantity': ['sum', 'mean', 'std', 'count']
}).reset_index()
item_demand.columns = ['item_id', 'total_qty', 'avg_qty', 'std_qty', 'order_count']
item_demand['cv'] = item_demand['std_qty'] / item_demand['avg_qty']  # Coefficient of variation

print("\nDemand Variability Analysis (Top 20 by volume):")
display(item_demand.nlargest(20, 'total_qty'))


Demand Variability Analysis (Top 20 by volume):


,item_id,total_qty,avg_qty,std_qty,order_count,cv
5582,480007,25642,2.880153,7.761231,8903,2.694729
11875,615418,23800,2.581345,2.308226,9220,0.894195
14184,653549,23501,1.183512,0.487416,19857,0.411839
14185,653567,19902,1.152536,0.442711,17268,0.384119
12870,622799,13085,1.987394,2.471713,6584,1.243696
17156,751704,12895,2.631633,2.576532,4900,0.979062
12343,618681,12561,1.467064,43.217913,8562,29.458783
12891,622975,12048,2.383854,2.210983,5054,0.927483
3138,429645,11764,2.520137,2.527208,4668,1.002806
3137,429643,11161,2.359619,2.199372,4730,0.932087


## 7. Places / Locations Analysis

In [16]:
# Analyze places
places = datasets['dim_places'].copy()

print(f"Total Places: {len(places)}")
print(f"\nColumns: {list(places.columns)}")

# Orders per place
orders_per_place = orders.groupby('place_id').size().reset_index(name='order_count')
top_places = orders_per_place.nlargest(10, 'order_count')

fig = px.bar(top_places, x='place_id', y='order_count',
             title='🏪 Top 10 Places by Order Volume',
             labels={'place_id': 'Place ID', 'order_count': 'Orders'},
             color='order_count',
             color_continuous_scale='Greens')
fig.update_layout(template='plotly_white')
fig.show()

Total Places: 1824

Columns: ['id', 'user_id', 'created', 'updated', 'title', 'activated', 'active', 'bankrupt', 'binding_period', 'chain_id', 'contact_email', 'contact_mobile_phone', 'contract_start', 'country', 'currency', 'cvr_number', 'duplicate', 'ecommerce_fee', 'eu_commission', 'invoicing_start_date', 'isv_commission', 'isv_partner', 'non_eu_commission', 'onboarded_by', 'payment_terminal_provider', 'area_id', 'processing_fee', 'sales_outcome_id', 'sales_stage', 'service_charge', 'setup_type', 'termination_date', 'termination_value', 'transferred_contract', 'type_id', 'cashier_ids', 'manager_ids', 'driver_ids', 'payment_terminal_ids', 'kitchen_staff_ids', 'contact_name', 'description', 'cuisine_ids', 'trainee_mode', 'demo_mode', 'default_order_type_cashier', 'default_order_type_customer', 'show_bestsellers', 'use_quick_search', 'display_cashier_images', 'display_created_by', 'use_customer_facing_display', 'timezone', 'contact_language', 'business_name', 'area', 'street_address', 

## 8. Key Findings Summary

In [17]:
print("""
╔══════════════════════════════════════════════════════════════╗
║                    📊 KEY FINDINGS SUMMARY                   ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  📈 SALES PATTERNS                                           ║
║  • Peak hours: [To be filled after analysis]                 ║
║  • Busiest days: [To be filled after analysis]               ║
║  • Seasonal trends: [To be filled after analysis]            ║
║                                                              ║
║  🏆 TOP PERFORMERS                                           ║
║  • Most ordered items identified                             ║
║  • Revenue drivers mapped                                    ║
║                                                              ║
║  📦 INVENTORY INSIGHTS                                       ║
║  • Demand variability calculated per item                    ║
║  • High-variance items flagged for safety stock              ║
║                                                              ║
║  🔮 FORECASTING POTENTIAL                                    ║
║  • Daily patterns suitable for Prophet/XGBoost               ║
║  • External factors (weekday/weekend) show impact            ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
""")


╔══════════════════════════════════════════════════════════════╗
║                    📊 KEY FINDINGS SUMMARY                   ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  📈 SALES PATTERNS                                           ║
║  • Peak hours: [To be filled after analysis]                 ║
║  • Busiest days: [To be filled after analysis]               ║
║  • Seasonal trends: [To be filled after analysis]            ║
║                                                              ║
║  🏆 TOP PERFORMERS                                           ║
║  • Most ordered items identified                             ║
║  • Revenue drivers mapped                                    ║
║                                                              ║
║  📦 INVENTORY INSIGHTS                                       ║
║  • Demand variability calculated per item                    ║
║  • High-variance items fla